In [99]:
# Installieren der benötigten Bibliotheken
# %pip install plotly pandas pynimate geopandas

In [100]:
# Laden der benötigten Bibliotheken
import pyarrow
import pandas as pd
import numpy as np
import plotly.express as px
from pynimate import Canvas, Barplot
import os
import json
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as tick
from matplotlib.animation import PillowWriter
import pynimate as nim
from pynimate.utils import human_readable

In [101]:
# Load from github
url = "https://github.com/MAD1982/dataviz_WS2526/raw/main/inkar/inkar_bayern_nordbayern.parquet"
df_inkar = pd.read_parquet(url)

In [102]:
### Datenexploration
df_inkar.info()
df_inkar.head()
df_inkar["Indikator"].drop_duplicates().sort_values()
df_inkar[["Name", "Kennziffer"]].drop_duplicates().sort_values("Kennziffer")
by_grosse_staedte = [
    "09161000", "09162000", "09362000",
    "09562000", "09563000", "09564000",
    "09663000", "09761000"
]

# Welche Städte sind noch in den Daten?
df_inkar = df_inkar[df_inkar["Kennziffer"].isin(by_grosse_staedte)]
df_inkar["Name"].drop_duplicates().sort_values()

# Welche Indikatoren sind in den Daten?
df_inkar["Indikator"].drop_duplicates().sort_values()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 850220 entries, 0 to 850219
Data columns (total 10 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Kennziffer  850220 non-null  string 
 1   Name        850220 non-null  object 
 2   Nordbayern  850220 non-null  boolean
 3   Raumbezug   850220 non-null  string 
 4   Zeitbezug   850220 non-null  string 
 5   Indikator   850124 non-null  object 
 6   Kuerzel     850220 non-null  object 
 7   Bereich     850028 non-null  object 
 8   ID          844940 non-null  string 
 9   Wert        850220 non-null  Float64
dtypes: Float64(1), boolean(1), object(4), string(4)
memory usage: 60.8+ MB


783995                 (SDG 1) Armut - Altersarmut
791568                 (SDG 1) Armut - Kinderarmut
785653               (SDG 1) SGB II-/SGB XII-Quote
746864    (SDG 10) Beschäftigungsquote - Ausländer
793230                     (SDG 10) Einbürgerungen
                            ...                   
845884                    Öffentliche Bibliotheken
847999                           Öffentliche Bäder
738072                        Öffentliche Finanzen
849548             Öffentliche Verwaltungsfunktion
850028                                        None
Name: Indikator, Length: 594, dtype: object

In [103]:
# Daten laden
df_inkar = pd.read_parquet(url)
V_Kategorie = "Offene Stellen mit Anforderungsniveau Fachkraft"
# Sortierung hoch oder runter
V_Sortierung = False
# Nordbayern filtern
df_inkar = df_inkar[df_inkar["Nordbayern"] == True]

# Nur Kreise
df_inkar = df_inkar[df_inkar["Raumbezug"] == "Kreise"]

# Daten filtern
indikatoren_liste = [V_Kategorie]
df_inkar = df_inkar[df_inkar["Indikator"].isin(indikatoren_liste)]

# Pivotieren der Daten in ein breites Format
df_wide = (
    df_inkar
    .loc[df_inkar["Indikator"].isin(indikatoren_liste)]
    .pivot_table(
        index=["Kennziffer", "Name", "Nordbayern", "Raumbezug", "Zeitbezug"],  # Keys je Beobachtung
        columns="Indikator",
        values="Wert",
        aggfunc="first"  # oder "mean" falls Duplikate existieren
    )
    .reset_index()
)

# optional: Spaltennamen "ent-flatten" (pivot_table erzeugt ggf. einen columns-Index)
df_wide.columns.name = None

# Lösche Zeilen in denen V_Kategorie NaN sind
df_wide = df_wide.dropna(subset=indikatoren_liste)

# Wieviele Kreise sollen angezeigt werden
TOP_N = 25

# Spalten sauber auswählen
df_plot = (
    df_wide[["Zeitbezug", "Name", V_Kategorie]]
    .rename(columns={
        "Zeitbezug": "Jahr",
        "Name": "Kreis/kreisfreie Stadt",
        V_Kategorie: V_Kategorie + " in Prozent",
    })
    .copy()
)

# Jahr numerisch
df_plot["Jahr"] = pd.to_numeric(df_plot["Jahr"], errors="coerce").astype("Int64")

# Werte numerisch (falls Strings/Kommas drin sind)
df_plot[V_Kategorie+" in Prozent"] = pd.to_numeric(
    df_plot[V_Kategorie+" in Prozent"], errors="coerce"
)

# NaN-Werte entfernen
df_plot = df_plot.dropna(subset=["Jahr", V_Kategorie+" in Prozent"])

# Optional: Top-N je Jahr (hier: höchste Ärztedichte = "best")
df_plot = (
    df_plot.sort_values(["Jahr", V_Kategorie+" in Prozent"], ascending=[True, False])
           .groupby("Jahr", as_index=False)
           .head(TOP_N)
)

# Sort-Key für Balkenreihenfolge
df_plot["sort_key"] = -df_plot[V_Kategorie+" in Prozent"]
df_plot = df_plot.sort_values(["Jahr", "sort_key"])

# Chart erstellen
fig = px.bar(
    df_plot,
    x=V_Kategorie+" in Prozent",
    y="Kreis/kreisfreie Stadt",
    color="Kreis/kreisfreie Stadt",
    animation_frame="Jahr",
    orientation="h",
    range_x=[0, float(df_plot[V_Kategorie+" in Prozent"].max()) * 1.05],
    title="Bar Chart Race:"+V_Kategorie+ " in Prozent"
)

# Sortierung
fig.update_layout(
    yaxis=dict(categoryorder="total ascending"),
    showlegend=True
)

fig.show()


In [104]:
# Wieviele Kreise sollen angezeigt werden
Top_N = 10

# welche Kategorie soll geplottet werden
V_Kategorie = "Offene Stellen mit Anforderungsniveau Fachkraft"
# Sortierung hoch oder runter
V_Sortierung = False
# Top oder Flop
Top = "Top-"
# Daten vorbereiten
df_plot = (
    df_wide[["Zeitbezug", "Name", V_Kategorie]]
    .rename(columns={
        "Zeitbezug": "Jahr",
        "Name": "Kreis/kreisfreie Stadt",
        V_Kategorie: V_Kategorie+" in Prozent",
    })
    .copy()
)

df_plot["Jahr"] = pd.to_numeric(df_plot["Jahr"], errors="coerce")
df_plot[V_Kategorie+" in Prozent"] = pd.to_numeric(
    df_plot[V_Kategorie+" in Prozent"], errors="coerce"
)
df_plot = df_plot.dropna(subset=["Jahr", V_Kategorie+" in Prozent"])
df_plot["Jahr"] = df_plot["Jahr"].astype(int)

# Top-10 Kreise bestimmen (nach letztem Jahr)
last_year = df_plot["Jahr"].max()

top10 = (
    df_plot[df_plot["Jahr"] == last_year]
    .sort_values(V_Kategorie+" in Prozent", ascending=V_Sortierung)
    .head(Top_N)["Kreis/kreisfreie Stadt"]
    .tolist()
)

df_top = df_plot[df_plot["Kreis/kreisfreie Stadt"].isin(top10)].copy()

# Optional: Facet-Reihenfolge nach Niveau im letzten Jahr sortieren
order = (
    df_top[df_top["Jahr"] == last_year]
    .sort_values(V_Kategorie+" in Prozent", ascending=True)["Kreis/kreisfreie Stadt"]
    .tolist()
)
df_top["Kreis/kreisfreie Stadt"] = pd.Categorical(
    df_top["Kreis/kreisfreie Stadt"], categories=order, ordered=True
)

# Small multiples: Facet je Kreis
fig = px.line(
    df_top,
    x="Jahr",
    y=V_Kategorie+" in Prozent",
    facet_col="Kreis/kreisfreie Stadt",
    facet_col_wrap=5,                           # 10 Kreise -> 2 Reihen à 5
    markers=True,
    title=("Small multiples: "+V_Kategorie+" in Prozent  "+ Top + str(Top_N) +"  (nach Jahr 2023)")
)

# Layout-Tuning
fig.update_layout(
    showlegend=False,
    margin=dict(l=20, r=20, t=70, b=20)
)

# Facet-Titel kürzen: "Kreis/kreisfreie Stadt=XYZ" -> "XYZ"
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))

# Einheitliche y-Achse über alle Facets
fig.update_yaxes(range=[0, 100], matches="y")
fig.update_yaxes(matches="y")


fig.show()


In [113]:
import pandas as pd
import plotly.graph_objects as go
from gapminder import gapminder
# Daten laden
df_inkar = pd.read_parquet(url)

# Sortierung hoch oder runter
V_Sortierung = False
# Nordbayern filtern
df_inkar = df_inkar[df_inkar["Nordbayern"] == True]

# Nur Kreise
df_inkar = df_inkar[df_inkar["Raumbezug"] == "Kreise"]

# Daten filtern
indikatoren_liste = ["Offene Stellen mit Anforderungsniveau Helfer",
"Offene Stellen mit Anforderungsniveau Fachkraft",
"Offene Stellen mit Anforderungsniveau Spezialist",
"Offene Stellen mit Anforderungsniveau Experte",

"Beschäftigte mit Anforderungsniveau Helfer",
"Beschäftigte mit Anforderungsniveau Fachkraft",
"Beschäftigte mit Anforderungsniveau Spezialist",
"Beschäftigte mit Anforderungsniveau Experte"]
df_inkar = df_inkar[df_inkar["Indikator"].isin(indikatoren_liste)]

# Pivotieren der Daten in ein breites Format
df_wide = (
    df_inkar
    .loc[df_inkar["Indikator"].isin(indikatoren_liste)]
    .pivot_table(
        index=["Kennziffer", "Name", "Nordbayern", "Raumbezug", "Zeitbezug"],  # Keys je Beobachtung
        columns="Indikator",
        values="Wert",
        aggfunc="first"  # oder "mean" falls Duplikate existieren
    )
    .reset_index()
)

# optional: Spaltennamen "ent-flatten" (pivot_table erzeugt ggf. einen columns-Index)
df_wide.columns.name = None

# Lösche Zeilen in denen V_Kategorie NaN sind
df_wide = df_wide.dropna(subset=indikatoren_liste)
# ----------------------------
# Daten laden & vorbereiten
# ----------------------------
df = df_wide


countries = sorted(df["Name"].unique())

# Variablen, die per Dropdown gewählt werden können
metrics = {
"Offene Stellen mit Anforderungsniveau Helfer":"Offene Stellen Helfer",
"Offene Stellen mit Anforderungsniveau Fachkraft":"Offene Stellen Fachkraft",
"Offene Stellen mit Anforderungsniveau Spezialist":"Offene Stellen Spezialist",
"Offene Stellen mit Anforderungsniveau Experte":"Offene Stellen Experte",

"Beschäftigte mit Anforderungsniveau Helfer":"Beschäftigte Helfer",
"Beschäftigte mit Anforderungsniveau Fachkraft":"Beschäftigte Fachkraft",
"Beschäftigte mit Anforderungsniveau Spezialist":"Beschäftigte Spezialist",
"Beschäftigte mit Anforderungsniveau Experte":"Beschäftigte Experte",
}

start_metric = "Offene Stellen mit Anforderungsniveau Helfer"
default_countries = ["Bamberg", "Amberg"]

# ----------------------------
# Daten pro Land & Variable vorbereiten
# ----------------------------
x_data = []  # Jahresachsen pro Land
data_per_metric = {m: [] for m in metrics.keys()}  # y-Daten pro Variable & Land

for country in countries:
    df_c = df[df["Name"] == country].sort_values("Zeitbezug")
    x_data.append(df_c["Zeitbezug"])
    for m in metrics.keys():
        data_per_metric[m].append(df_c[m])

# ----------------------------
# Figure initialisieren – ein Trace pro Land
# ----------------------------
fig = go.Figure()

for i, country in enumerate(countries):
    # Nur Germany & Austria initial einblenden, Rest über Legende wählbar
    visible_state = True if country in default_countries else "legendonly"
    fig.add_trace(
        go.Scatter(
            x=x_data[i],
            y=data_per_metric[start_metric][i],
            mode="lines+markers",
            name=country,
            visible=visible_state,
            hovertemplate=(
                "Kreis: " + country +
                "<br>Jahr: %{x|%Y}" +
                "<br>Wert: %{y:.2f}<extra></extra>"
            )
        )
    )

# ----------------------------
# Dropdown-Menü für Variable
# ----------------------------
metric_buttons = []
for metric_key, metric_label in metrics.items():
    metric_buttons.append(
        dict(
            label=metric_label,
            method="update",
            args=[
                # y-Daten aller Traces auf die gewählte Variable setzen
                {"y": data_per_metric[metric_key]},
                # Achsentitel & Plot-Titel anpassen
                {
                    "yaxis": {"title": metric_label},
                    "title": f"{metric_label} in ausgewählten Kreise"
                }
            ]
        )
    )

# ----------------------------
# Layout + Slider + Selector
# ----------------------------
fig.update_layout(
    title=f"{metrics[start_metric]} in ausgewählten Kreise",
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(
                    count=20,
                    label="letzte 20 Jahre",
                    step="year",
                    stepmode="backward"
                ),
                dict(step="all", label="alle")
            ])
        ),
        rangeslider=dict(visible=True),
        type="date",
        title="Jahr"
    ),
    yaxis=dict(
        title=metrics[start_metric]
    ),
    updatemenus=[
        dict(
            type="dropdown",
            buttons=metric_buttons,
            x=1.15,
            y=1.0,
            xanchor="left",
            yanchor="top",
            showactive=True
        )
    ],
    height=500
)

fig.show()
